# AUTOENCODER
This file have purpose to anlysys performance of autoencoder model

reference : https://towardsdatascience.com/generating-new-faces-with-variational-autoencoders-d13cfcb5f0a8
## Prepare Program and Data

In [1]:
# Import library create by mysel
from library.directory_handle import DirectoryHandle
import library.image_handle as ImageHandle
import library.data_handle as DataHandle

# Import library for plot image
import matplotlib.pyplot as plt

# Import library for manage model part Core Layers
from keras.layers import Input, Flatten, Dense, Reshape
# Import library for manage model part Convolution Layers
from keras.layers import Conv2D, Conv2DTranspose
# Import Library for manage model part Model Object
from keras.models import Model
# Import Library for manage model part optimizer
from keras.optimizers import Adam
# Import Library about model 
from keras.utils import plot_model
# Import library for load model
from keras.models import load_model

#Import library for normal process
import numpy as np

Using TensorFlow backend.


In [2]:
# Constant data path to collect directory data
PATH_DATA = "/home/zeabus/Documents/supasan/2019_deep_learning/PokemonData" 

# Part parameter in normal program will require to input
crop = True
color = True
rounds = 10 # Round to train data (epoches)

In [3]:
# Process Read data 
directory_handle = DirectoryHandle( PATH_DATA )
list_label , list_data = directory_handle.group_data()
list_dictionary = directory_handle.group_dictionary()

smallest_size = ImageHandle.min_all_square_size1( list_data )
smallest_size = smallest_size if smallest_size % 2 == 0 else smallest_size - 1
latent_size = int( smallest_size * smallest_size / 4 )
latent_size = latent_size * 3 if color else latent_size
print(f'Will manage by resize image to smallest size {smallest_size} square and have latent size {latent_size}')

input_dim = ( smallest_size , smallest_size , 3 if color else 1 )

X_data , Y_data = ImageHandle.prepare_label_data( list_label , list_data , smallest_size , color = color , crop = crop )

Will manage by resize image to smallest size 42 square and have latent size 1323


In [4]:
# Before train data first time must split data for train and test
(X_train,Y_train) , (X_test,Y_test) = DataHandle.train_test_split( X_data , Y_data , 8 )
print(f'From {len(X_data)} datas split for train {len(X_train)} and test {len(X_test)}')

From 6781 datas split for train 5934 and test 847


## Model Part
Above part is part prepare data for train test and analysis

In [ ]:
# Set up model part encoder
encoder_input = Input( shape = input_dim , name = 'encode_input' )
encoder = encoder_input
encoder = Conv2D( filters = 64,
                kernel_size = (3,3),
                strides=2,
                padding='same',
                activation='relu',
                name = "encoder_convolution_1")(encoder)
encoder = Conv2D( filters = 32,
                kernel_size = (3,3),
                strides=1,
                padding='same',
                activation='relu',
                name = "encoder_convolution_2")(encoder)
encoder = Conv2D( filters = 16,
                kernel_size = (3,3),
                strides=1,
                padding='same',
                activation='relu',
                name = "encoder_convolution_3")(encoder)
shape_before_flatten = (int(smallest_size/2) , int(smallest_size/2) , 16)
encoder = Flatten()(encoder)
encoder_output = Dense( latent_size , name = "encoder_output")(encoder)

encoder_model = Model( encoder_input , encoder_output )
encoder_model.name = "encoder_model"
encoder_model.summary()

In [ ]:
decoder_input = Input( shape = (latent_size,) , name ='decoder_input')
decoder = Dense( np.prod(shape_before_flatten) , name='decoder_input_convolution' )(decoder_input)
decoder = Reshape( shape_before_flatten , name='decoder_input_reshape')(decoder)
decoder = Conv2DTranspose( filters = 32,
                        kernel_size = (3,3),
                        strides=1,
                        padding='same',
                        name='decoder_convolution_1',
                        activation='relu')(decoder)
decoder = Conv2DTranspose( filters = 64,
                        kernel_size = (3,3),
                        strides=2,
                        padding='same',
                        name='decoder_convolution_2',
                        activation='relu')(decoder)
decoder_output = Conv2DTranspose( filters = 3 if color else 1,
                        kernel_size = (3,3),
                        strides=1,
                        padding='same',
                        name='decoder_output',
                        activation='relu')(decoder)
decoder_model = Model( decoder_input , decoder_output )
decoder_model.name = "decoder_model"
decoder_model.summary()

In [ ]:
autoencoder = Model( encoder_input , decoder_model(encoder_output) )
autoencoder.summary()

## Train Model

In [ ]:
optimizer = Adam( lr = 0.0005 ) # set learning
autoencoder.compile( optimizer=optimizer,
                    loss='mean_squared_error',
                    metrics=['accuracy'])
history = autoencoder.fit( [X_train], # Input
                          [X_train], # Output
                         validation_data=( [X_test] , [X_test]),
                         epochs = rounds )

In [ ]:
# Plot traing & validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
autoencoder.save( "pokemon_autoencoder.h5")

## Result Model

In [ ]:
autoencoder = load_model( "pokemon_autoencoder.h5" )

In [ ]:
random_index = []
for _ in range(0,10):
    random_index.append( np.random.randint( len( X_test)))
random_index = tuple( set( random_index ) )
data = []
for index in random_index :
    data.append( X_test[ index ] )
data = np.array( data )

In [ ]:
data[0].shape

In [ ]:
DataHandle.plot_compare( data , autoencoder )

# Part Classifier
Above part is model for autoencoder below part will push Fully Connected Layer for classifier

In [ ]:
classifier_input = Input( shape = (latent_size,) , name = "classifier_input" )
classifier = Dense( int( latent_size / 2 ), 
        activation='relu',
        name = 'classifier_network_1' )( classifier_input )
classifier_output = Dense( len(list_label),
        activation='softmax',
        name = 'classifier_output' )( classifier )

classifier_model = Model( classifier_input , classifier_output )
classifier_model.name = "fully_connected"
classifier_model.summary()

In [ ]:
classifier = Model( encoder_input , classifier_model( encoder_output ) )
classifier.name = "classifier_model"
optimizer = Adam( lr = 0.0005 ) # set learning
classifier.compile( optimizer=optimizer,
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
classifier.summary()

In [ ]:
# Next we will save weights
for run in range( 0 , len( classifier.layers ) - 1 ):
    classifier.layers[ run ].set_weights( autoencoder.layers[run].get_weights() )
    classifier.layers[ run ].trainable = False
classifier.summary()

In [ ]:
history = classifier.fit( [X_train], # Input
                          [Y_train], # Output
                         validation_data=( [X_test] , [Y_test]),
                         epochs = rounds )

In [ ]:
classifier.save( "pokemon_autoencoder_classifier.h5")

## Result Model Classifier on AutoEncoder

In [5]:
classifier = load_model( "pokemon_autoencoder_classifier.h5" )
predict = classifier.predict( [X_test] )

In [6]:
DataHandle.result_classifier( predict , np.array( Y_test ) , list_dictionary )

Report classifier system 847 datas
Summary correct 124.0 datas from 847.0 datas
Name                     |      PRECISION |         RECALL
-------------------------------------------------------------------------------
Alakazam                 |        0.00000 |        0.00000
Diglett                  |        0.00000 |        0.00000
Tauros                   |        0.11111 |        0.16667
Poliwrath                |        0.00000 |        0.00000
Jynx                     |        0.75000 |        0.50000
Machop                   |        0.25000 |        0.33333
Abra                     |        0.00000 |        0.00000
Voltorb                  |        0.55556 |        0.71429
Golbat                   |        0.00000 |        0.00000
Magneton                 |        0.00000 |        0.00000
Gloom                    |        0.33333 |        0.33333
Alolan Sandslash         |        0.00000 |        0.00000
Kangaskhan               |        0.00000 |        0.00000
Horsea        

/home/zeabus/Documents/supasan/2019_deep_learning/deep_learning_project/src_classifier/data_handle.py:77: RuntimeWarning: invalid value encountered in double_scalars
  precision[ run ] = correct[run] / precision[ run ]
